In [1]:
from dotenv import load_dotenv
from libs.docx_parser import getDoc
from libs.docx_parser import getCsv
from langchain import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
import os

In [2]:
# Cargo variables de configuración
load_dotenv()

True

In [ ]:
# Este código se ejecuta cada vez que desea generar el csv de los Excel de los CP.
#from libs.convert_excel_to_csv import convert_xlsx_to_csv

#convert_xlsx_to_csv('documentos/redenciones/cp_redenciones.xlsx', 'Casos de Prueba', 'documentos/redenciones/cp_redenciones.csv', 5, 8, 'Unnamed: 2', 'Unnamed: 7')

In [3]:
# Carga documento input y tabla objetivo.
#context_file1 = 'documentos/migraciones/cp_migraciones.docx'
#context1 = getDoc(context_file1)

#targte_file1 = 'documentos/migraciones/objetivos_migraciones.csv'
#target1 = getCsv(targte_file1)

# Carga documento input y tabla objetivo.
context_file2 = 'documentos/redenciones/cp_redenciones.docx'
context2 = getDoc(context_file2)

targte_file2 = 'documentos/redenciones/cp_redenciones.csv'
target2 = getCsv(targte_file2)

# Carga documento input y tabla objetivo.
#context_file3 = 'documentos/msi/cp_msi.docx'
#context3 = getDoc(context_file3)

#targte_file3 = 'documentos/msi/objetivos_msi.csv'
#target3 = getCsv(targte_file3)

In [5]:
prompt = """You will be provided with a context from which you should extract multiple test cases by identifying the following aspects:
 'Caso de prueba', 'Modulo', 'Funcionalidad', 'Proceso' and 'Segmento'.
   As the text contains multiple test cases, the output should be presented in a csv format table.
   Here is an example:"""

In [6]:
examples = [
    {
        "context": context2,
        "result": target2
    } 
]

In [36]:
"""examples = [
    {
        "context": context1,
        "result": target1
    },
    {
        "context": context2,
        "result": target2
    },
    {
        "context": context3,
        "result": target3
    }
]"""

'examples = [\n    {\n        "context": context1,\n        "result": target1\n    },\n    {\n        "context": context2,\n        "result": target2\n    },\n    {\n        "context": context3,\n        "result": target3\n    }\n]'

In [7]:
example_template = """
Context: {context}
Result: {result}
"""


example_prompt = PromptTemplate(
    input_variables=["context", "result"],
    template=example_template
)


prefix = """Sos parte del equipo de testing re una compania de telecomunicaciones.
Te voy a dar un documento con requerimientos de testeo para uno de los modulos de la aplicacion.
Necesito que hagas una lista con todos los casos de prueba que puedas identificar.
 Deberás incluir la condicion esperada para cada uno de los casos.
    
   Aqui hay ejemplos:
   """

suffix = """
Context: {new_context}
Result: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["new_context"],
    example_separator="\n\n"
)


In [8]:
#contexto_path = 'documentos/migraciones/hu_migraciones.docx'
contexto_path = 'documentos/redenciones/hu_redenciones.docx'
#contexto_path = 'documentos/msi/hu_msi.docx'
nuevo_contexto = getDoc(contexto_path)
#print(nuevo_contexto)

In [9]:
llm = AzureChatOpenAI (
        azure_endpoint=os.getenv('API_BASE'),
        openai_api_version=os.getenv('API_VERSION'),
        deployment_name=os.getenv('CHAT_ENGINE_16K'),
        openai_api_key=os.getenv('API_KEY'),
        openai_api_type=os.getenv('API_TYPE'),
        temperature=0,
        model_version="0613" # es importante aclararlo para el correcto calculo de los costos
        )

llm_chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

result = llm_chain(nuevo_contexto)

In [10]:
result['text']

'1. Prueba de ingreso al mantenedor con el usuario NBA en la sección Parrillas flujos nativos, Pestaña Fidelización y Crear bloque.\n   - Condición esperada: Creación exitosa del bloque.\n\n2. Prueba de ingreso al mantenedor con el usuario NBA en la sección reportes y redención.\n   - Condición esperada: Generación exitosa del reporte.\n\n3. Prueba de ingreso a la aplicación con un DN apto para redención y selección de beneficio.\n   - Condición esperada: Flujo de redención exitoso y pantalla de Success.\n\n4. Prueba de ingreso a la aplicación con el DN 5537685200 y selección de beneficio.\n   - Condición esperada: Flujo de redención exitoso y pantalla de Success.\n\n5. Prueba de ingreso al mantenedor en la sección Parrillas y creación de bono, regalo o descuento.\n   - Condición esperada: Creación exitosa del bono, regalo o descuento.\n\n6. Prueba de ingreso al mantenedor en la sección Parrillas y configuración de la campaña de fidelización.\n   - Condición esperada: Configuración exi

In [11]:
# guardo el resultado de result['text] en un archvivo .csv
with open('documentos/resultados/redenciones_fs.csv', 'w') as f:
    f.write(result['text'])